In [ ]:
!pip install jsonlines

In [ ]:
import jsonlines
import string 
import pandas as pd
import json
import glob
import random
import re

#Spacy imports
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from spacy import displacy
from collections import Counter
from spacy.util import minibatch, compounding
from spacy.util import decaying

import warnings
import pickle
import plac
from pathlib import Path
from tqdm import tqdm

spacy.prefer_gpu()

In [ ]:
#reading all json files
lst = []
file = r"E:\Lawnics\Spacy Acts and Cases\output1.jsonl"
with jsonlines.open(file) as reader:
    for obj in reader:
        lst.append(obj)
len(lst)

In [ ]:
#creating the dataset (which will be cleaned in further steps)
Dataset = []
for data in lst:
    ents = [tuple(entity) for entity in data['labels']]
    Dataset.append((data['text'].lower().strip(),{'entities':ents}))
random.shuffle(Dataset)
Dataset = Dataset[0:2000]
test_dataset = Dataset[1500:2000]
print(len(Dataset))

In [ ]:
# def punc(text):
#     # define punctuation
#     punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

#     my_str = text

#     # To take input from the user
#     # my_str = input("Enter a string: ")

#     # remove punctuation from the string
#     no_punct = ""
#     for char in my_str:
#         if char not in punctuations:
#             no_punct = no_punct + char

#     # display the unpunctuated string
#     return no_punct
# new_lst = []
# for item in Dataset:
#     new_lst.append([item[0],item[1]])
# for i in tqdm(new_lst):
#     i[0] = punc(i[0])
# new_lst[0]
# Dataset = []
# for j in new_lst:
#     Dataset.append((j[0],j[1]))


In [ ]:
Dataset[110]

In [ ]:
#creating a dictionary of the annotations
def create_dict(x):
    dictionary = {}
    text = x[0]
    annot = x[1]
    index = annot['entities']
    for start,end,tag in index:
        word = text[start:end].lower().strip()
        dictionary[word]=tag
    return dictionary

In [ ]:
dictionary = {}
for val in Dataset:
    tmp_dict = create_dict(val)
#     print(tmp_dict)
#     break
    dictionary.update(tmp_dict)
    
    
dictionary = {k.strip().lower(): v for (k, v) in dictionary.items()}
len(dictionary)
dictionary

In [ ]:
for keys in dictionary.copy():
    if len(keys) <= 4:
        dictionary.pop(keys)
    if "act" in keys:
        dictionary[keys] = "Act"
    if "vs" in keys:
        dictionary[keys] = "Cases"
dictionary["ipc"] = "Act"
dictionary["indian penal code"] = "Act"
dictionary["code of criminal procedure"] = "Act"
dictionary["code of civil procedure"] = "Act"
dictionary["the criminal procedure code"] = "Act"
dictionary["cr.pc"] = "Act"
dictionary["cr.p.c"] = "Act"
# dictionary.pop("free for one month")
for keys in dictionary.copy():
    if dictionary[keys] == "Other":
        dictionary.pop(keys)

for k in dictionary.copy():
    if "act" in k:
        dictionary[k] = "Act"
#     if dictionary[k] == "Cases":
#         res= re.split(r'(^[^\d]+)', k)
#         if len(res) > 1:
#             res = res[1]
#             dictionary[res] = "Cases"
# dictionary

In [ ]:
# Dataset[0][0]

In [ ]:
#Checking if overlapping annotations exist. For example "Judgement and udgement(wrongly annotated)" or "Supreme Court and"
def is_overlap(a, b):
    a, b = sorted([a, b])
    if a[0] <= b[0] and a[1] >= b[1]:
        return True
    elif a[1] > b[0]:
        return True
    return False

In [ ]:
# assert is_overlap((1,9), (3,4)) == True
# assert is_overlap((3,4), (1,9)) == True
# assert is_overlap((1, 5), (3, 7)) == True
# assert is_overlap((4, 9), (1, 6)) == True
# assert is_overlap((1,6), (7,9)) == False
# assert is_overlap((1,4),(4,6)) == False

In [ ]:
# dictionary

In [ ]:
# [k for k in dictionary if len(k) == 44]

In [ ]:
# from collections import Counter
# Counter(map(len, dictionary)).most_common(10)

In [ ]:
# sorted(dictionary.items(), key=lambda x: len(x[0]), reverse=True)[:10]

In [ ]:
import re

In [ ]:
#Script to clean out the annotations
dictionary2 = {re.compile(re.escape(k)): v for k,v in sorted(dictionary.items(), key=lambda x: len(x[0]), reverse=True) if len(k) >= 4}

In [ ]:
#Script to re-annotate the dataset
from __future__ import unicode_literals, print_function
def entities_finder(document):  
    entities = []
    my_str=document
    some_list= set()
    for key, final_tag in dictionary2.items():
        start = 0
        for match in key.finditer(my_str):
            out_interval = match.span()
            if out_interval in some_list:
                continue
            if not any(is_overlap(out_interval, interval) for interval in some_list):
                entities.append((out_interval[0], out_interval[1], final_tag))
                some_list.add(out_interval)

    return entities

In [ ]:
#To store in the format specified for Spacy
def formatting_func(text):
    val=(text,{'entities':entities_finder(text)})
    return val

In [ ]:
cleaned_dataset=[]
for i in tqdm(sorted(Dataset, key=lambda x: len(x[0]))):
    cleaned_dataset.append(formatting_func(i[0]))

In [ ]:

for item in tqdm(cleaned_dataset):
    lst = []
    for ent in item[1]["entities"]:
        if ent[2] != "Cases":
            lst.append(ent)
        if ent[2] == "Cases":
            st = item[0][ent[0]:ent[1]] 
            if st != "others":
                if "v" in st or "vs" in st:
                    if "penal" not in st:
                        lst.append(ent)
    item[1]["entities"] = lst 

In [ ]:
ent

In [ ]:
for ent in item[1]["entities"]:
        if ent[2] == "Cases":
            st = item[0][ent[0]:ent[1]] 
            print(st)

In [ ]:
for item in tqdm(cleaned_dataset):
    for n in item[1]["entities"]:
        if n[2] == "Other":
            print(item)

In [ ]:
# len(sorted(Dataset, key=lambda x: len(x[0]),reverse=True)[3][0])

In [ ]:
# len(sorted(Dataset, key=lambda x: len(x[0]),reverse=True)[0][0])

In [ ]:
cleaned_dataset[0]

In [ ]:
random.shuffle(cleaned_dataset)

In [ ]:
train_set = cleaned_dataset[:200]
len(train_set)

In [ ]:
test_set = cleaned_dataset[500:1000]
len(test_set)

In [ ]:
def evaluate(ner_model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = ner_model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot['entities'])
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

In [ ]:
train_set[0]

In [ ]:
#!/usr/bin/env python
# coding: utf8

# Training additional entity types using spaCy

warnings.simplefilter("ignore", UserWarning)
# LABEL = ['Authority', 'Action', 'Area', 'Party', 'Subject', 'SubjectElements'] #currently used annotations
LABEL = ['Section', 'Article', 'Cases', 'Act'] #currently used annotations
def main(model=None, new_model_name='new_model', output_dir=r"E:\Lawnics\Spacy Acts and Cases", n_iter=5):
    """Setting up the pipeline and entity recognizer, and training the new entity."""
    max_batch_size = 16
    if len(train_set) < 500:
        max_batch_size /= 2
    nlp = spacy.blank('en')  # create blank Language class
    print("Created blank 'en' model")
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner)
        nlp.add_pipe(nlp.create_pipe('sentencizer'))
    else:
        ner = nlp.get_pipe('ner')

    for i in LABEL:
        ner.add_label(i)   # Add new entity labels to entity recognizer

    if model is None:
        optimizer = nlp.begin_training()
    else:
        optimizer = nlp.entity.create_optimizer()

    # Get names of other pipes to disable them during training to train only NER
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        for itn in range(n_iter):
            random.shuffle(train_set)
            losses = {}T
            batches = minibatch(train_set, size=compounding(1, max_batch_size, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.5,
                           losses=losses)
            scores = evaluate(nlp, train_set)
            print("Scores ",scores)
            print('Losses', losses)
                # Save model 
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.meta['name'] = new_model_name  # rename model
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)
            
prdnlp = main()

In [ ]:
# to render the generated annotations
nlp2 = spacy.load(r"E:\Lawnics\Spacy Acts and Cases")
doc = nlp2(test_set[100][0])
displacy.render(nlp2(str(doc)), jupyter=True, style='ent')

In [ ]:
from spacy.gold import docs_to_json
doc = nlp2(test_dataset[150][0])
json_data = docs_to_json([doc])

In [ ]:
json_data

In [ ]:
#Evaluate the model on the test_set created after auto-annotations
examples = test_set[10:50]
results = evaluate(nlp2, examples)
print(results)

In [ ]:
# This checks for mis-alignments in the annotations shown by ('-'). Spacy ignores these as a Userwarning
# But too many mis-alignments will lead to a crunch in your train set

from spacy.gold import biluo_tags_from_offsets
for text, annot in train_set:
    doc = nlp2.make_doc(text)
    print(biluo_tags_from_offsets(doc, annot["entities"]))

In [ ]:
import spacy
from spacy import displacy
# from blackstone.displacy_palette import ner_displacy_options

nlp = spacy.load('en_core_web_sm')

# text = """
# The applicant must satisfy a high standard. This is a case where the action is to be tried by a judge with a jury. The standard is set out in Jameel v Wall Street Journal Europe Sprl [2004] EMLR 89, para 14:
# “But every time a meaning is shut out (including any holding that the words complained of either are, or are not, capable of bearing a defamatory meaning) it must be remembered that the judge is taking it upon himself to rule in effect that any jury would be perverse to take a different view on the question. It is a high threshold of exclusion. Ever since Fox’s Act 1792 (32 Geo 3, c 60) the meaning of words in civil as well as criminal libel proceedings has been constitutionally a matter for the jury. The judge’s function is no more and no less than to pre-empt perversity. That being clearly the position with regard to whether or not words are capable of being understood as defamatory or, as the case may be, non-defamatory, I see no basis on which it could sensibly be otherwise with regard to differing levels of defamatory meaning. Often the question whether words are defamatory at all and, if so, what level of defamatory meaning they bear will overlap.”
# 18 In Berezovsky v Forbes Inc [2001] EMLR 1030, para 16 Sedley LJ had stated the test this way:
# “The real question in the present case is how the courts ought to go about ascertaining the range of legitimate meanings. Eady J regarded it as a matter of impression. That is all right, it seems to us, provided that the impression is not of what the words mean but of what a jury could sensibly think they meant. Such an exercise is an exercise in generosity, not in parsimony.”
# """
text = Dataset[100][0]

doc = nlp(text)

# Call displacy and pass `ner_displacy_options` into the option parameter`
displacy.render(doc, style="ent")

In [ ]:
# import spacy
import spacy
from spacy.matcher import Matcher
# load english language model
nlp = spacy.load('en_core_web_sm',disable=['ner','textcat'])

# text = "This is a sample sentence."

# create spacy 
doc = nlp(text)

for token in doc:
#     print(token.text,'->',token.pos_)
    if token.pos_ == "NOUN":
        print(token.text)

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

In [ ]:
test_str = "asdkjah other"
lst = ["1","2","3","4","5","6","7","8","9"]
res = re.split(r'(^[^\d]+)', test_str)
res[1]

In [ ]:
dct = {"l":"other"}

In [ ]:
for k in dct:
    if dct[k] == "other":
        dct[k] = "Cases"
dct